# FEWSNET Historical View Experiments
This notebook explores how we can combine FEWSNET report data. The goal would be to have one dataset with polygons and another that controls the color of the polygons based on date selected.

In [3]:
import geopandas as gpd
import pandas as pd
from requests.auth import HTTPBasicAuth
import requests
import json
import plotly.express as px
import os
import rtree

In [50]:
# This will read data from FUSENET directly. Could be a good route to go in the future.
# testlink = gpd.read_file("https://fews.net/data_portal_download/download?data_file_path=http://shapefiles.fews.net.s3.amazonaws.com/HFIC/EA/east-africa201802.zip")
# testlink = testlink.to_crs(epsg=4326)
# testlink

In [4]:
# All livelihood Zones
lhz = gpd.read_file("./data/FEWS_NET_LH_World.zip")
lhz = lhz.to_crs(epsg=4326)
lhz

,OBJECTID,FNID,EFF_YEAR,COUNTRY,LZNUM,LZCODE,LZNAMEEN,LZNAMEFR,LZNAMESP,LZNAMEPT,CLASS,LZSZCODE,LZTYPE,MAINCROPS,MAINLSTOCK,geometry
0,30,AO2013L102,2013,AO,2,AO02,Transitional banana and pineapple farming,None,None,Transicional do cultivo da banana e ananás,AS09,None,None,None,None,"POLYGON ((14.32935 -10.30132, 14.35352 -10.313..."
1,31,AO2013L104,2013,AO,4,AO04,Sub humid livestock and maize,None,None,Sub húmida de pastorícia e milho,PS03,None,None,None,None,"POLYGON ((17.89424 -13.67606, 17.90491 -13.695..."
2,32,AO2013L105,2013,AO,5,AO05,Mid-eastern cassava and savannah forest,None,None,Centro-este de mandioca e floresta de savana,AG04,None,None,None,None,"POLYGON ((17.92018 -11.52576, 17.93213 -11.527..."
3,33,AO2013L108,2013,AO,8,AO08,"Transitional lowland maize, cassava, and beans",None,None,"Baixa transicional de milho, mandioca e feijões",AS05,None,None,None,None,"POLYGON ((13.83407 -7.65854, 13.83546 -7.68132..."
4,34,AO2013L110,2013,AO,10,AO10,Savannah forest and market-oriented cassava,None,None,Floresta de savana e mandioca orientada ao mer...,AN08,None,None,None,None,"POLYGON ((17.11363 -7.42542, 17.12791 -7.42627..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,0,TD2021L104,2021,TD,4,TD04,Southeast Flood Retreat and Gum Arabic,Zone sud-est: cultures de décrue et gomme arab...,None,None,AN01,None,None,None,None,"POLYGON ((22.47709 12.01926, 22.47845 12.01923..."
861,0,TD2021L103,2021,TD,3,TD03,South-central Cereals,Zone sud-centrale céréaliculture,None,None,AN04,None,None,None,None,"POLYGON ((19.40499 11.10400, 19.47900 11.04799..."
862,0,TD2021L102,2021,TD,2,TD02,Southwest Rice,Zone sud-ouest de riz \r,None,None,AS05,None,None,None,None,"POLYGON ((15.39857 9.77331, 15.39858 9.77331, ..."
863,0,TD2021L105,2021,TD,5,TD05,Central Agropastoral,Zone centrale agropastorale,None,None,PS03,None,None,None,None,"POLYGON ((17.27100 13.66200, 17.23400 13.42700..."


In [5]:
# All Admin 1 zones
adm1 = gpd.read_file("./data/FEWSNET_World_Admin/FEWSNET_Admin1.shp")
adm1 = adm1.to_crs(epsg=4326)
adm1

,FNID,EFF_YEAR,COUNTRY,ADMIN0,ADMIN1,ADMIN1FR,ADMIN1SP,ADMIN1PT,ALIASES,geometry
0,AO2008A101,2008,AO,Angola,Bengo,None,None,Bengo,None,"POLYGON ((13.99846 -7.65368, 13.99921 -7.65384..."
1,AO2008A102,2008,AO,Angola,Benguela,None,None,Benguela,None,"POLYGON ((13.87370 -11.76882, 13.87458 -11.768..."
2,AO2008A103,2008,AO,Angola,Bie,None,None,Bie,None,"POLYGON ((16.61016 -10.57761, 16.61522 -10.578..."
3,AO2008A104,2008,AO,Angola,Cabinda,None,None,Cabinda,None,"POLYGON ((12.82282 -4.40323, 12.83247 -4.40642..."
4,AO2008A105,2008,AO,Angola,Cunene,None,None,Cunene,None,"POLYGON ((16.54543 -15.13035, 16.54644 -15.130..."
...,...,...,...,...,...,...,...,...,...,...
664,AF2017A122,2017,AF,Afghanistan,Sari Pul,None,None,None,~Sar-e-Pul~Sar-e Pul~,"POLYGON ((66.00733 36.50566, 66.00871 36.50549..."
665,AF2017A118,2017,AF,Afghanistan,Takhar,None,None,None,~Tekhar~Taluqan~Talukan~,"MULTIPOLYGON (((70.06893 37.38423, 70.06896 37..."
666,AF2017A125,2017,AF,Afghanistan,Uruzgan,None,None,None,~Oruzgan~Owruzgan~Urozgan~Orozgan~,"POLYGON ((66.52982 33.59141, 66.53017 33.58970..."
667,AF2017A104,2017,AF,Afghanistan,Wardak,None,None,None,~Maydan~Vardak~Werdek~Wardag~,"POLYGON ((68.29716 34.78276, 68.29839 34.78109..."


In [6]:
# All Admin 2 zones
adm2 = gpd.read_file("./data/FEWSNET_World_Admin/FEWSNET_Admin2.shp")
adm2 = adm2.to_crs(epsg=4326)
adm2

,FNID,EFF_YEAR,COUNTRY,ADMIN0,ADMIN1,ADMIN1FR,ADMIN1SP,ADMIN1PT,ADMIN2,ADMIN2FR,ADMIN2SP,ADMIN2PT,ALIASES,geometry
0,AO2008A21701,2008,AO,Angola,Uige,None,None,Uige,Alto Cuale,None,None,Alto Cuale,None,"POLYGON ((16.34542 -7.47550, 16.34498 -7.47693..."
1,AO2008A21501,2008,AO,Angola,Moxico,None,None,Moxico,Alto Zambeze,None,None,Alto Zambeze,None,"POLYGON ((24.01645 -11.01787, 24.01387 -11.030..."
2,AO2008A20901,2008,AO,Angola,Kuanza Norte,None,None,Kuanza Norte,Ambaca,None,None,Ambaca,None,"POLYGON ((15.69911 -8.01768, 15.70128 -8.01780..."
3,AO2008A21001,2008,AO,Angola,Kuanza Sul,None,None,Kuanza Sul,Amboim,None,None,Amboim,None,"POLYGON ((14.51918 -10.76831, 14.52180 -10.770..."
4,AO2008A20101,2008,AO,Angola,Bengo,None,None,Bengo,Ambriz,None,None,Ambriz,None,"POLYGON ((13.81469 -7.66262, 13.81628 -7.66297..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,AF2017A20105,2017,AF,Afghanistan,Kabul,None,None,None,Deh Sabz,None,None,None,~Dehsabz~Deh-e-Sabz~Deh-e Sabz~,"POLYGON ((69.46155 34.58448, 69.46121 34.58369..."
6561,AF2017A20907,2017,AF,Afghanistan,Baghlan,None,None,None,Andarab,None,None,None,None,"POLYGON ((69.04545 35.81928, 69.04450 35.81818..."
6562,AF2017A21713,2017,AF,Afghanistan,Badakhshan,None,None,None,Shahr-e-Buzorg,None,None,None,~Shahr-e Buzurg~Shahri Buzurg~Shahr-e Buzurg~,"POLYGON ((70.06893 37.38423, 70.07026 37.38434..."
6563,AF2017A20102,2017,AF,Afghanistan,Kabul,None,None,None,Paghman,None,None,None,None,"POLYGON ((69.03724 34.59171, 69.03838 34.59173..."


In [7]:
# Example of a mini dataset
minidf = gpd.read_file("./data/ALL_HFIC/East Africa/EA_201802_CS.shp")
minidf = minidf.to_crs(epsg=4326)
minidf.head(20)

,CS,HA0,geometry
0,1.0,0.0,"MULTIPOLYGON (((39.36784 -4.65351, 39.36945 -4..."
1,1.0,1.0,"MULTIPOLYGON (((42.78954 3.30069, 42.71921 3.2..."
2,2.0,0.0,"MULTIPOLYGON (((39.54055 -3.63975, 39.54312 -3..."
3,2.0,1.0,"MULTIPOLYGON (((44.45677 2.13468, 44.44286 2.1..."
4,3.0,0.0,"MULTIPOLYGON (((36.80593 -1.59263, 36.80575 -1..."
5,3.0,1.0,"MULTIPOLYGON (((31.37783 6.50423, 31.47383 6.3..."
6,4.0,0.0,"MULTIPOLYGON (((30.12183 6.35244, 30.15537 6.2..."
7,88.0,0.0,"MULTIPOLYGON (((29.67210 -0.88200, 29.67230 -0..."
8,99.0,0.0,"MULTIPOLYGON (((46.98970 8.00000, 47.01239 8.0..."


In [8]:
# Example of a large dataset
largedf = gpd.read_file("./data/ALL_HFIC/East Africa/EA_202110_CS.shp")
largedf = largedf.to_crs(epsg=4326)
largedf

,cov_start,cov_end,report_mon,country,unit_name,ADMIN0,ADMIN1,ADMIN2,ADMIN3,LZCODE,LZNAME,CS,HA0,geometry
0,10-2021,10-2021,10-2021,BI,"Buragane, Makamba, Burundi",Burundi,Makamba,None,None,BI01,Buragane,1,0,"POLYGON ((29.79122 -4.01030, 29.79211 -4.01100..."
1,10-2021,10-2021,10-2021,BI,"Buragane, Rutana, Burundi",Burundi,Rutana,None,None,BI01,Buragane,1,0,"POLYGON ((29.97427 -3.97512, 29.97411 -3.97570..."
2,10-2021,10-2021,10-2021,BI,"Congo Ridge Millet, Bubanza, Burundi",Burundi,Bubanza,None,None,BI02,Congo Ridge Millet,1,0,"POLYGON ((29.39921 -2.96037, 29.39955 -2.96100..."
3,10-2021,10-2021,10-2021,BI,"Congo Ridge Millet, Bujumbura Rural, Burundi",Burundi,Bujumbura Rural,None,None,BI02,Congo Ridge Millet,1,0,"POLYGON ((29.50249 -3.26355, 29.50681 -3.26968..."
4,10-2021,10-2021,10-2021,BI,"Congo Ridge Millet, Bururi, Burundi",Burundi,Bururi,None,None,BI02,Congo Ridge Millet,1,0,"POLYGON ((29.64906 -4.10597, 29.64796 -4.10726..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2599,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Arua, ...",Uganda,Northern,Arua,None,UG15,None,2,0,"MULTIPOLYGON (((31.20456 3.33572, 31.20450 3.3..."
2600,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Koboko...",Uganda,Northern,Koboko,None,UG15,None,2,0,"POLYGON ((31.10081 3.74161, 31.10047 3.73584, ..."
2601,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Madi O...",Uganda,Northern,Madi Okollo,None,UG15,None,2,0,"MULTIPOLYGON (((31.07543 2.82913, 31.07725 2.8..."
2602,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Marach...",Uganda,Northern,Maracha,None,UG15,None,2,0,"POLYGON ((31.04593 3.32229, 31.04595 3.32224, ..."


In [9]:
# goes through each folder and creates a dictionary with file name and column names
# there is also a list created with just the file names for use later
# It adds each of the files with more metadata to the large_files list

large_files = []

# Central America files
cam_dict = {}
cam_files = os.listdir("./data/ALL_HFIC/Central America and the Caribbean")
for file in cam_files:
    if file.endswith(".shp"):
        tempgdf = gpd.read_file("./data/ALL_HFIC/Central America and the Caribbean/" + file)
        cam_dict[file] = tempgdf.columns
        if "ADMIN1" in tempgdf.columns:
            large_files.append(file)

# Central Asia files
cas_dict = {}
cas_files = os.listdir("./data/ALL_HFIC/Central Asia")
for file in cas_files:
    if file.endswith(".shp"):
        tempgdf = gpd.read_file("./data/ALL_HFIC/Central Asia/" + file)
        cas_dict[file] = tempgdf.columns
        if "ADMIN1" in tempgdf.columns:
            large_files.append(file)

# East Africa files
ea_dict = {}
ea_files = os.listdir("./data/ALL_HFIC/East Africa")
for file in ea_files:
    if file.endswith(".shp"):
        tempgdf = gpd.read_file("./data/ALL_HFIC/East Africa/" + file)
        ea_dict[file] = tempgdf.columns
        if "ADMIN1" in tempgdf.columns:
            large_files.append(file)

# West Africa files
wa_dict = {}
wa_files = os.listdir("./data/ALL_HFIC/West Africa")
for file in wa_files:
    if file.endswith(".shp"):
        tempgdf = gpd.read_file("./data/ALL_HFIC/West Africa/" + file)
        wa_dict[file] = tempgdf.columns
        if "ADMIN1" in tempgdf.columns:
            large_files.append(file)

# South Africa files
sa_dict = {}
sa_files = os.listdir("./data/ALL_HFIC/Southern Africa")
for file in sa_files:
    if file.endswith(".shp"):
        tempgdf = gpd.read_file("./data/ALL_HFIC/Southern Africa/" + file)
        sa_dict[file] = tempgdf.columns
        if "ADMIN1" in tempgdf.columns:
            large_files.append(file)


In [10]:
large_files

['LAC_201107_CS.shp',
 'LAC_200910_CS.shp',
 'LAC_201204_CS.shp',
 'LAC_201304_CS.shp',
 'LAC_201410_CS.shp',
 'CAaC_202110_CS.shp',
 'EA_202110_CS.shp',
 'WA_202110_CS.shp',
 'SA_202110_CS.shp']

At this point we have dictionaries of files in each part of the world. There are files with more metadata for every region besides central asia.
There a couple weird datasets in central asia, but we will ignore them for now.

In [44]:
inter_adm2 = adm2.overlay(minidf, how='intersection')
# Getting rid of 88 and 99 values
inter_adm2 = inter_adm2[inter_adm2["CS"] != 99]
inter_adm2 = inter_adm2[inter_adm2["CS"] != 88]
inter_adm2.head(2)

/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:2352: UserWarning:

`keep_geom_type=True` in overlay resulted in 1330 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries



,FNID,EFF_YEAR,COUNTRY,ADMIN0,ADMIN1,ADMIN1FR,ADMIN1SP,ADMIN1PT,ADMIN2,ADMIN2FR,ADMIN2SP,ADMIN2PT,ALIASES,CS,HA0,geometry
0,CF2003A25203,2003,CF,Central African Republic,Haute-Kotto,Haute-Kotto,None,None,Yalinga,Yalinga,None,None,None,1.0,0.0,"MULTIPOLYGON (((23.52011 8.72697, 23.53575 8.7..."
1,CF2003A25301,2003,CF,Central African Republic,Vakaga,Vakaga,None,None,Birao,Birao,None,None,None,1.0,0.0,"MULTIPOLYGON (((23.27061 10.48983, 23.26384 10..."


In [15]:
eminidf = minidf[minidf["CS"] != 99]
eminidf = eminidf[eminidf["CS"] != 88]

In [43]:
inter_lhz = lhz.overlay(minidf, how='intersection')
# Getting rid of 88 and 99 values
inter_lhz = inter_lhz[inter_lhz["CS"] != 99]
inter_lhz = inter_lhz[inter_lhz["CS"] != 88]
inter_lhz.head(1)

/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:2352: UserWarning:

`keep_geom_type=True` in overlay resulted in 262 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries



,OBJECTID,FNID,EFF_YEAR,COUNTRY,LZNUM,LZCODE,LZNAMEEN,LZNAMEFR,LZNAMESP,LZNAMEPT,CLASS,LZSZCODE,LZTYPE,MAINCROPS,MAINLSTOCK,CS,HA0,geometry
0,61,CF2012L101,2012,CF,1,CF01,"Southeast cassava, peanuts, and maize","Sud-est manioc, arachide et maïs",None,None,AG12,None,None,None,None,2.0,0.0,"MULTIPOLYGON (((27.28350 5.58545, 27.28382 5.5..."


## Exploring joined on livelihood zones

The joined data compared to the original data looks good. See maps below. 
### Original Livelihood zones
![](./charts/lhzoriginal.png)
### Joined Livelihood zones
![](./charts/lhzjoined.png)

What the charts below shows is that many of the FNID's are repeated in the joined dataset the histogram below shows that in the original Livelihood zones there is only one FNID that is repeated and that one is only repeated once. FNID SS2018L107 is repeated 6 times in the join, but only once in the original Livelihood zones data. The below map shows what this looks like from a geographic perspective.

In [47]:
# plot distribution of OBJECTID in lhz_full and lhz
fig = px.histogram(lhz, x="FNID")
fig.show()
# plot distribution of OBJECTID in lhz_full and lhz
fig1 = px.histogram(inter_lhz, x="FNID")
fig1.show()

In [55]:
# making a dataframe of rows of inter_lhz that have FNID=SS2018L107 so I can plot and compare to the livelihood zone map
lhz_107 = inter_lhz[inter_lhz["FNID"] == "SS2018L107"]
# mapped this but deleted the cell for the sake of size of notebook

Map of original livelihood zone compared to the joined file shows that the joined files have finer resolution results than the original when livelihood zones are concerned.

### Original livelihood zone
![](./charts/originalmap.png)
### Joined livelihood zone
![](./charts/joinedmap.png)

## Exploring joined on admin2 zones

The joined on admin2 data compared to the original data looks good. See maps below. 
### Original Livelihood zones
![](./charts/lhzoriginal.png)
### Joined admin2 zones
![](./charts/admin2joined.png)

What the charts below shows is the same thing we found in the livelihood zones joined. Many of the FNID's are repeated in the joined dataset. the histogram below shows that almost all FNIDs in the adm2 dataset are not repeated. In the admin2 joined data many are repeated up to six times  (e.g. SS2011A21111).

In [57]:
# plot distribution of OBJECTID in lhz_full and lhz
fig = px.histogram(inter_adm2, x="FNID")
fig.show()
# plot distribution of OBJECTID in lhz_full and lhz
fig1 = px.histogram(adm2, x="FNID")
fig1.show()

In [74]:
# making a dataframe of rows of inter_lhz that have FNID=SS2018L107 so I can plot and compare to the livelihood zone map
adm_ss2 = inter_adm2[inter_adm2["FNID"] == "SO1990A22001"]
original_ss2 = adm2[adm2["FNID"] == "SO1990A22001"]
# mapped this but deleted the cell for the sake of size of notebook

The below screenshots show, similarly to livelihood zones that the joined data has finer resolution than the original data.

### Original admin2 data
![](./charts/adminoriginal.png)
### Joined admin2 data
![](./charts/adminjoined.png)

In [77]:
# plots geodataframe on mapbox map with hover. commented out for size of notebook
# mapdf = adm_ss2
# fig = px.choropleth_mapbox(mapdf, geojson=mapdf.geometry, locations=mapdf.index, color="CS", hover_name="CS", hover_data=["CS"], center={"lat": 0, "lon": 0}, mapbox_style="carto-positron", zoom=1, opacity=0.5)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

## Path forward
1. I'm going to try to spatially join past food security ratings to the recent very detailed report to see if the very detailed reports could be the "main boundaries" for measurements. Previously I was using their livelihood zone and admin2 datasets as "main boundaries"
2. If 1 doesn't work I'm going to create master dataset of polygons IPC values and dates. I think that the data is manageable enough that we could still create a historical view

## Joining with detailed dataset
I'm going to try to spatially join past food security ratings to the recent very detailed report to see if the very detailed reports could be the "main boundaries" for measurements. Previously I was using their livelihood zone and admin2 datasets as "main boundaries"

In [9]:
minidf

,CS,HA0,geometry
0,1.0,0.0,"MULTIPOLYGON (((39.36784 -4.65351, 39.36945 -4..."
1,1.0,1.0,"MULTIPOLYGON (((42.78954 3.30069, 42.71921 3.2..."
2,2.0,0.0,"MULTIPOLYGON (((39.54055 -3.63975, 39.54312 -3..."
3,2.0,1.0,"MULTIPOLYGON (((44.45677 2.13468, 44.44286 2.1..."
4,3.0,0.0,"MULTIPOLYGON (((36.80593 -1.59263, 36.80575 -1..."
5,3.0,1.0,"MULTIPOLYGON (((31.37783 6.50423, 31.47383 6.3..."
6,4.0,0.0,"MULTIPOLYGON (((30.12183 6.35244, 30.15537 6.2..."
7,88.0,0.0,"MULTIPOLYGON (((29.67210 -0.88200, 29.67230 -0..."
8,99.0,0.0,"MULTIPOLYGON (((46.98970 8.00000, 47.01239 8.0..."


In [12]:
largedf

,cov_start,cov_end,report_mon,country,unit_name,ADMIN0,ADMIN1,ADMIN2,ADMIN3,LZCODE,LZNAME,CS,HA0,geometry
0,10-2021,10-2021,10-2021,BI,"Buragane, Makamba, Burundi",Burundi,Makamba,None,None,BI01,Buragane,1,0,"POLYGON ((29.79122 -4.01030, 29.79211 -4.01100..."
1,10-2021,10-2021,10-2021,BI,"Buragane, Rutana, Burundi",Burundi,Rutana,None,None,BI01,Buragane,1,0,"POLYGON ((29.97427 -3.97512, 29.97411 -3.97570..."
2,10-2021,10-2021,10-2021,BI,"Congo Ridge Millet, Bubanza, Burundi",Burundi,Bubanza,None,None,BI02,Congo Ridge Millet,1,0,"POLYGON ((29.39921 -2.96037, 29.39955 -2.96100..."
3,10-2021,10-2021,10-2021,BI,"Congo Ridge Millet, Bujumbura Rural, Burundi",Burundi,Bujumbura Rural,None,None,BI02,Congo Ridge Millet,1,0,"POLYGON ((29.50249 -3.26355, 29.50681 -3.26968..."
4,10-2021,10-2021,10-2021,BI,"Congo Ridge Millet, Bururi, Burundi",Burundi,Bururi,None,None,BI02,Congo Ridge Millet,1,0,"POLYGON ((29.64906 -4.10597, 29.64796 -4.10726..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2599,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Arua, ...",Uganda,Northern,Arua,None,UG15,None,2,0,"MULTIPOLYGON (((31.20456 3.33572, 31.20450 3.3..."
2600,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Koboko...",Uganda,Northern,Koboko,None,UG15,None,2,0,"POLYGON ((31.10081 3.74161, 31.10047 3.73584, ..."
2601,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Madi O...",Uganda,Northern,Madi Okollo,None,UG15,None,2,0,"MULTIPOLYGON (((31.07543 2.82913, 31.07725 2.8..."
2602,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Marach...",Uganda,Northern,Maracha,None,UG15,None,2,0,"POLYGON ((31.04593 3.32229, 31.04595 3.32224, ..."


In [ ]:
inter_large = largedf.overlay(minidf, how='intersection')

In [15]:
# Getting rid of 88 and 99 values
inter_large = inter_large[inter_large["CS_1"] != 99]
inter_large = inter_large[inter_large["CS_1"] != 88]
inter_large = inter_large[inter_large["CS_2"] != 99]
inter_large = inter_large[inter_large["CS_2"] != 88]
inter_large

,cov_start,cov_end,report_mon,country,unit_name,ADMIN0,ADMIN1,ADMIN2,ADMIN3,LZCODE,LZNAME,CS_1,HA0_1,CS_2,HA0_2,geometry
0,10-2021,10-2021,10-2021,ET,"Abaala, Kilbati /Zone2, Afar, Ethiopia",Ethiopia,Afar,Kilbati /Zone2,Abaala,None,None,4,0,1.0,0.0,"MULTIPOLYGON (((39.79974 13.22378, 39.79906 13..."
1,10-2021,10-2021,10-2021,ET,"Abaala town, Kilbati /Zone2, Afar, Ethiopia",Ethiopia,Afar,Kilbati /Zone2,Abaala town,None,None,4,0,1.0,0.0,"MULTIPOLYGON (((39.76732 13.28527, 39.76266 13..."
2,10-2021,10-2021,10-2021,ET,"Ababo, Horo Gudru Wellega, Oromia, Ethiopia",Ethiopia,Oromia,Horo Gudru Wellega,Ababo,None,None,1,0,1.0,0.0,"POLYGON ((37.39828 10.04615, 37.40515 10.04259..."
3,10-2021,10-2021,10-2021,ET,"Abadir, Harari, Harari, Ethiopia",Ethiopia,Harari,Harari,Abadir,None,None,2,0,1.0,0.0,"POLYGON ((42.08530 9.32904, 42.08523 9.32906, ..."
4,10-2021,10-2021,10-2021,ET,"Abay Chomen, Horo Gudru Wellega, Oromia, Ethiopia",Ethiopia,Oromia,Horo Gudru Wellega,Abay Chomen,None,None,1,0,1.0,0.0,"POLYGON ((37.43733 9.61566, 37.43665 9.61556, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3736,10-2021,10-2021,10-2021,SD,"Rainfed Sorghum Belt, Buram, South Darfur, Sudan",Sudan,South Darfur,Buram,None,SD11,None,2,0,4.0,0.0,"MULTIPOLYGON (((25.42439 10.36786, 25.42439 10..."
3737,10-2021,10-2021,10-2021,SD,"Southeast Rainfed Semi-mechanized Agriculture,...",Sudan,South Kordofan,Abu Jubayhah,None,SD10,None,2,0,4.0,0.0,"MULTIPOLYGON (((32.47063 11.04272, 32.46091 11..."
3738,10-2021,10-2021,10-2021,SD,"Southeast Rainfed Semi-mechanized Agriculture,...",Sudan,Blue Nile,Al Kurmuk,None,SD10,None,2,0,4.0,0.0,"MULTIPOLYGON (((34.11325 9.49864, 34.11272 9.4..."
3739,10-2021,10-2021,10-2021,SD,"Southeast Rainfed Semi-mechanized Agriculture,...",Sudan,White Nile,As Salam / Ar Rawat,None,SD10,None,3,1,4.0,0.0,"MULTIPOLYGON (((32.74523 11.94928, 32.74512 11..."


Same issues exist as stated in joins above. Boundaries seem to be unique to each report. See map screenshots below.
### original mini dataframe
![](./charts/originalminidf.png)
### Joined with Large dataframe
![](./charts/largejoined.png)
what this shows is not all regions are measured when assessing just the geography from the larger dataset in the grouping